### SETUP INICIAL DO PROJETO

In [1]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account


# carregar o .env com as credenciais
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")


# detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"


# parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.production-productcategory"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"


# configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


#doc: tables_to_process: lista de tabelas que serão processadas
#     output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned

In [2]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

Tabelas a processar: ['desafioadventureworks-446600.raw_data.production-productcategory']


In [3]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

raw_data
Tabelas disponíveis:
humanresources-employee
person-address
person-businessentity
person-person
person-stateprovince
production-location
production-product
production-productcategory
production-productcosthistory
production-productinventory
production-productsubcategory
purchasing-purchaseorderdetail
purchasing-purchaseorderheader
purchasing-vendor
sales-creditcard
sales-customer
sales-salesorderdetail
sales-salesorderheader
sales-salesperson
sales-salesterritory
sales-store


# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [4]:
# Setup inicial do df para realizar a EDA 

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [5]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")


print("Todas as tabelas foram processadas com sucesso!")

Processando tabela: desafioadventureworks-446600.raw_data.production-productcategory
Lendo os dados do BigQuery...
Tabela production_productcategory processada e armazenada com sucesso.
Todas as tabelas foram processadas com sucesso!


In [6]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

Variável criada: production_productcategory


In [7]:
# Atribuir o df a uma variável com nome mais simples
production_productcategory = df_processados['production_productcategory']

print(f"Colunas: {production_productcategory.shape[1]}\nLinhas: {production_productcategory.shape[0]}")

Colunas: 4
Linhas: 52


In [8]:
# Identificar duplicadas
duplicadas = production_productcategory[production_productcategory.duplicated(subset=['productcategoryid'], keep=False)]

# Verificar se existem duplicadas
if not duplicadas.empty:
    
    duplicadas_ordenadas = duplicadas.sort_values(by=['productcategoryid', 'modifieddate'])

    print("duplicadas ordenadas:")
    print(duplicadas_ordenadas)
else:
    print("Não foram encontradas duplicadas.")

duplicadas ordenadas:
    productcategoryid         name                               rowguid              modifieddate
0                   1        Bikes  cfbda25c-df71-47a7-b81b-64ee161aa37c 2008-04-30 00:00:00+00:00
1                   1        Bikes  cfbda25c-df71-47a7-b81b-64ee161aa37c 2008-04-30 00:00:00+00:00
2                   1        Bikes  cfbda25c-df71-47a7-b81b-64ee161aa37c 2008-04-30 00:00:00+00:00
3                   1        Bikes  cfbda25c-df71-47a7-b81b-64ee161aa37c 2008-04-30 00:00:00+00:00
4                   1        Bikes  cfbda25c-df71-47a7-b81b-64ee161aa37c 2008-04-30 00:00:00+00:00
5                   1        Bikes  cfbda25c-df71-47a7-b81b-64ee161aa37c 2008-04-30 00:00:00+00:00
6                   1        Bikes  cfbda25c-df71-47a7-b81b-64ee161aa37c 2008-04-30 00:00:00+00:00
7                   1        Bikes  cfbda25c-df71-47a7-b81b-64ee161aa37c 2008-04-30 00:00:00+00:00
8                   1        Bikes  cfbda25c-df71-47a7-b81b-64ee161aa37c 2008-04-30 00:

In [9]:
# Remover duplicadas* 
production_productcategory = production_productcategory.drop_duplicates(subset=['productcategoryid'], keep='last')

print(f"Linhas após remover duplicadas (baseando-se na última 'modifieddate'): {len(production_productcategory)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicadas = production_productcategory.copy()


#doc: bkp = backup (cópia)
#doc*: mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
#      Importante, pois se houver erro na ingestão (duplicação), mantém os dados íntegros.

Linhas após remover duplicadas (baseando-se na última 'modifieddate'): 4


In [10]:
# Ordenar e exibir o df por 'productcategoryid'
production_productcategory = production_productcategory.sort_values(by=['productcategoryid'])

print(production_productcategory)

    productcategoryid         name                               rowguid              modifieddate
48                  1        Bikes  cfbda25c-df71-47a7-b81b-64ee161aa37c 2008-04-30 00:00:00+00:00
49                  2   Components  c657828d-d808-4aba-91a3-af2ce02300e9 2008-04-30 00:00:00+00:00
50                  3     Clothing  10a7c342-ca82-48d4-8a38-46a2eb089b74 2008-04-30 00:00:00+00:00
51                  4  Accessories  2be3be36-d9a2-4eee-b593-ed895d97c2a6 2008-04-30 00:00:00+00:00


In [11]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in production_productcategory.columns:   
    missing_rows = production_productcategory[production_productcategory[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

Coluna 'productcategoryid': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'productcategoryid'.

Coluna 'name': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'name'.

Coluna 'rowguid': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'rowguid'.

Coluna 'modifieddate': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'modifieddate'.



In [12]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = production_productcategory.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

Valores únicos incluindo NaN:
productcategoryid    4
name                 4
rowguid              4
modifieddate         1
dtype: int64


In [13]:
#verificar informações do df
production_productcategory.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 48 to 51
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   productcategoryid  4 non-null      Int64              
 1   name               4 non-null      object             
 2   rowguid            4 non-null      object             
 3   modifieddate       4 non-null      datetime64[us, UTC]
dtypes: Int64(1), datetime64[us, UTC](1), object(2)
memory usage: 164.0+ bytes


In [14]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['production_productcategory'] = production_productcategory

In [15]:
# Padronizar colunas com valores textuais*
production_productcategory['name'] = production_productcategory['name'].str.strip().str.upper()
production_productcategory['rowguid'] = production_productcategory['rowguid'].str.strip().str.upper()

print(production_productcategory.head())

#doc*: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

    productcategoryid         name                               rowguid              modifieddate
48                  1        BIKES  CFBDA25C-DF71-47A7-B81B-64EE161AA37C 2008-04-30 00:00:00+00:00
49                  2   COMPONENTS  C657828D-D808-4ABA-91A3-AF2CE02300E9 2008-04-30 00:00:00+00:00
50                  3     CLOTHING  10A7C342-CA82-48D4-8A38-46A2EB089B74 2008-04-30 00:00:00+00:00
51                  4  ACCESSORIES  2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6 2008-04-30 00:00:00+00:00


In [16]:
# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
 
    output_table = f"{output_dataset}.{table_name}"
   
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  
    )
    
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")

Tabela production_productcategory exportada com sucesso para desafioadventureworks-446600.raw_data_cleaned.production_productcategory.
